In [1]:
import czifile as zis
from apeer_ometiff_library import io, processing, omexmlClass
import os

In [2]:
basefolder = r'/datadisk1/tuxedo/IPython_Notebooks/testdata'

#filename_ometiff = os.path.join(basefolder, 'Filter_with_Particles_big.ome.tiff')
filename_ometiff = os.path.join(basefolder, 'S=2_T=5_CH=3_CH=2_A2.ome.tiff')
#filename_czi = os.path.join(basefolder, 'Filter_with_Particles_big.czi')
#filename_czi = os.path.join(basefolder, '8Brains_DAPI_5X_stitched.czi')
#filename_czi = os.path.join(basefolder, 'S=2_T=5_Z=3_CH=2_A2.czii')

In [3]:
def create_metadata_dict():
    """
    A Python dictionary will be created to hold the relevant metadata.
    """

    metadata = {'Directory': '',
                'Filename': '',
                'Name': '',
                'AcqDate': '',
                'TotalSeries': 0,
                'SizeX': 0,
                'SizeY': 0,
                'SizeZ': 0,
                'SizeC': 0,
                'SizeT': 0,
                'DimOrder BF': 'n.a.',
                'NA': 0,
                'ObjMag': 0,
                'ObjID': 'n.a.',
                'XScale': 0,
                'YScale': 0,
                'ZScale': 0,
                'XScaleUnit': 'n.a.',
                'YScaleUnit': 'n.a.',
                'ZScaleUnit': 'n.a.',
                'DetectorModel': [],
                'DetectorName': [],
                'DetectorID': None,
                'InstrumentID': None,
                'Channels': [],
                'ImageIDs': []
               }

    return metadata

def get_metadata(filename, omexml,
                 series=0):
    
    # create dictionary for metadata and get OME-XML data
    metadata = create_metadata_dict()
    md = omexmlClass.OMEXML(omexml)
    
    # get directory and filename etc.
    metadata['Directory'] = os.path.dirname(filename)
    metadata['Filename'] = os.path.basename(filename)
    metadata['AcqDate'] = md.image(series).AcquisitionDate
    metadata['Name'] = md.image(series).Name
    
    # get image dimensions
    metadata['SizeT'] = md.image(series).Pixels.SizeT
    metadata['SizeZ'] = md.image(series).Pixels.SizeZ
    metadata['SizeC'] = md.image(series).Pixels.SizeC
    metadata['SizeX'] = md.image(series).Pixels.SizeX
    metadata['SizeY'] = md.image(series).Pixels.SizeY
    metadata['TotalSeries'] = md.get_image_count()
    
    # get dimension order
    metadata['DimOrder BF'] = md.image(series).Pixels.DimensionOrder
    
    # get the scaling
    metadata['XScale'] = md.image(series).Pixels.PhysicalSizeX
    metadata['XScaleUnit'] = md.image(series).Pixels.PhysicalSizeXUnit
    metadata['YScale'] = md.image(series).Pixels.PhysicalSizeY
    metadata['YScaleUnit'] = md.image(series).Pixels.PhysicalSizeYUnit
    metadata['ZScale'] = md.image(series).Pixels.PhysicalSizeZ
    metadata['ZScaleUnit'] = md.image(series).Pixels.PhysicalSizeZUnit
    
    # get all image IDs
    for i in range(md.get_image_count()):
        metadata['ImageIDs'].append(i)

    # get information baout the instrument and objective
    metadata['InstrumentID'] = md.instrument(series).get_ID()
    metadata['DetectorModel'] = md.instrument(series).Detector.get_Model()
    metadata['DetectorID'] = md.instrument(series).Detector.get_ID()
    metadata['DetectorModel'] = md.instrument(series).Detector.get_Type()
    metadata['NA'] = md.instrument(series).Objective.get_LensNA()
    metadata['ObjID'] = md.instrument(series).Objective.get_ID()
    metadata['ObjMag'] = md.instrument(series).Objective.get_NominalMagnification()
    
    # get channel names
    for c in range(metadata['SizeC']):
        metadata['Channels'].append(md.image(series).Pixels.Channel(c).Name)
        
    return metadata

In [4]:
image_name = os.path.basename(filename_ometiff)

if filename_ometiff.lower().endswith('.ome.tiff') or filename_ometiff.lower().endswith('.ome.tif'):
    
    # Return value is an 5D Array of order (T, Z, C, X, Y)
    (array5d, omexml) = io.read_ometiff(filename_ometiff)
    
metadata = get_metadata(filename_ometiff, omexml, series=0)

In [5]:
array5d.shape

(5, 3, 2, 1216, 1216)

In [6]:
metadata

{'Directory': '/datadisk1/tuxedo/IPython_Notebooks/testdata',
 'Filename': 'S=2_T=5_CH=3_CH=2_A2.ome.tiff',
 'Name': 'S=2_T=5_Z=3_CH=2_A2.czi #1',
 'AcqDate': '2014-08-27T10:51:22.039',
 'TotalSeries': 1,
 'SizeX': 1216,
 'SizeY': 1216,
 'SizeZ': 3,
 'SizeC': 2,
 'SizeT': 5,
 'DimOrder BF': 'XYCZT',
 'NA': '0.025',
 'ObjMag': '1.0',
 'ObjID': 'Objective:1',
 'XScale': 1.0,
 'YScale': 1.0,
 'ZScale': 3.0,
 'XScaleUnit': 'µm',
 'YScaleUnit': 'µm',
 'ZScaleUnit': 'µm',
 'DetectorModel': None,
 'DetectorName': [],
 'DetectorID': 'Detector:Camera',
 'InstrumentID': 'Instrument:0',
 'Channels': ['DAPI', 'EGFP'],
 'ImageIDs': [0]}